<a href="https://colab.research.google.com/github/nadienuncanada/BigData/blob/main/BigData_TP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# BigData 2024

## Inicializacion

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
ruta = '/content/drive/MyDrive/BigData'
ruta_MRE = ruta + '/MRE.py'
GENERALinputPath= ruta + '/Tp1/input'
outputPath = './output'

import os

# Definir el nombre de la carpeta

def crearcarpeta(carpeta):
  # Crear la carpeta si no existe
  if not os.path.exists(carpeta):
      os.makedirs(carpeta)
      print(f'Carpeta "{carpeta}" creada con éxito.')
  else:
      print(f'La carpeta "{carpeta}" ya existe.')

import sys
sys.path.append(ruta)

from MRE import Job

Mounted at /content/drive


## Trabajo Practico 1





### Datos del enunciado


**Pautas generales**

• La entrega consiste en la implementación de Jobs MapReduce, resolviendo todas las
consignas presentes en este enunciado. Se deberá entregar el código fuente
implementado y un documento con el DAG del proceso completo.

• Los alumnos pueden conformar grupo de no más de dos integrantes y hacer una única
entrega grupal.

• La entrega se realiza por la mensajería del curso en IDEAS.

• La fecha límite de entrega es el 28 de octubre de 2024.

**Aclaración**

Suponga que la cantidad de jugadores, de misiones y de héroes es Big Data.
En cada job planteado en la solución piense si una función combiner contribuye o no para la
optimización del job. En caso de contribuir implemente dicha función

**Enunciado**

Una empresa ha desarrollado un videojuego online donde millones de jugadores se conectan a
diario para realizar diferentes misiones. Cada jugador elije una misión en la cual participar y de
su participación obtiene un puntaje y el tiempo en segundo que duró la misión. El jugador para
participar de una misión debe elegir cinco héroes de un conjunto de héroes disponibles. Por
cada participación, los servidores del juego almacenan la siguiente información:


• ID_Jugador

• ID_Misión

• ID_Héroe_1

• ID_Héroe_2

• ID_Héroe_3

• ID_Héroe_4

• ID_Héroe_5

• Puntaje obtenido

• Tiempo de la misión en segundos

Donde los cinco ID de los héroes son IDs de un conjunto de héroes que el jugador tiene para
elegir y no guardan relación alguna entre ellos.
Todos los jugadores pueden participar en la misión que deseen cuántas veces deseen y por cada
misión obtienen un puntaje y también se registra el tiempo que duró la misión. Un jugador
puede jugar todas las veces que quiera.
Cada vez que un jugador decide jugar una misión, debe elegir cinco héroes, los cuales pueden
ser totalmente distintos de una partida a la siguiente, aún jugando en la misma misión.
Por ejemplo, el jugador con ID 231 juega a la misión 492 con los héroes 42, 901, 72, 831 y 4.
Luego el mismo jugador puede jugar a la misma misión con otros héroes o los mismos, o repetir
solo algunos de ellos.

Semanalmente los administradores del juego quieren obtener algunas estadísticas para
regalarle premios a los mejores jugadores y hacer mejoras al juego según la preferencia de los
jugadores. Al finalizar la semana se desea saber:


### EJ1
La misión más jugada (independientemente de la cantidad de jugadores que la jugaron,
ni cuántas veces la jugó cada uno).

In [ ]:
#Job 1: Cuenta la cant de repeticiones por mision

inputPath = GENERALinputPath
outputPath = './output'
crearcarpeta("output")

def fmap(key, value, context):
  datos = value.split()
  context.write(datos[0], 1)

def freduce(key,values,context):
  cont = 0
  for v in values:
    cont+=v
  context.write(key,cont)

job = Job(inputPath, outputPath, fmap, freduce)
job.setCombiner(freduce) #EL COMBINER SI CONTRIBUYE => Contamos antes de trasnferir, por lo que trasferimos menos datos
job.waitForCompletion()

Carpeta "output" creada con éxito.


True

In [ ]:
#Job 2: Obtiene el max

crearcarpeta("output2")
outputPath2 = './output2'

inputPath = outputPath

def fmap(key, value, context):
  context.write("", (key,value))

def freduce(key,values,context):
  max = 0
  mision = None
  for v in values:
    valor=int(v[1])
    if(valor>max):
      max=valor
      mision=v[0]
  context.write(mision,max)

job = Job(inputPath, outputPath2, fmap, freduce)
job.setCombiner(freduce) #EL COMBINER SI CONTRIBUYE => Obtenemos el maximo parcial antes de trasnferir, por lo que trasferimos menos datos
job.waitForCompletion()

La carpeta "output2" ya existe.


True

### EJ2
El héroe más elegido en cada una de las misiones (el primer puesto podría ser
compartido por más de un héroe, de hecho, todos los héroes podrían haber participado
de la misión la misma cantidad de veces).


In [ ]:
#Job 1: Cuenta la cantidad de repeticiones de la tupla (mision, heroe)

inputPath = GENERALinputPath
outputPath = './output'
crearcarpeta("output")

def fmap(key, value, context):
  datos = value.split()
  for i in range(1,6):
    heroe = datos[i]
    context.write((datos[0] , datos[i]), 1)

def freduce(key,values,context):
  cont = 0
  for v in values:
    cont+=v
  context.write(key,cont)

job = Job(inputPath, outputPath, fmap, freduce)
job.setCombiner(freduce) #EL COMBINER SI CONTRIBUYE => Contamos antes de trasnferir, por lo que trasferimos menos datos
job.waitForCompletion()

La carpeta "output" ya existe.


True

In [ ]:
#Job 2: Conseguir que heroe/s se jugaron mayor cantidad de veces por cada mision. Tenemos que segurarnos de que el reduce le llegue primero el maximo

inputPath = outputPath

outputPath2 = './output2'
crearcarpeta("output2")


def fmap(key, value, context):
  datos=value.split()
  context.write((key,datos[1]),datos)#id_mision+cantidad-> como key, queremos la cantidad como key para ordernalo en el sort

def fShuffle(aKey, anotherKey):#juntamos por mision
 if(aKey[0] == anotherKey[0]):
  return 0
 elif(aKey[0] < anotherKey[0]):
  return -1
 else:
  return 1

def fSort(aKey, anotherKey):#ordenamos por cantidad descendente
 if(int(aKey[1]) == int(anotherKey[1])):
  return 0
 elif(int(aKey[1]) < int(anotherKey[1])):
  return 1
 else:
  return -1

def freduce(key, value, context):
  max=-1
  for v in value:
    cant=int(v[1])
    if(int(max)<=cant):
      max = cant
    if(max!=cant):
      break
    context.write(key[0],(v[0],max))# id_m, id_h cantidad.

job = Job(inputPath, outputPath2, fmap, freduce)


job.setShuffleCmp(fShuffle)
job.setSortCmp(fSort)

#ACA COMBINER NO CONTRIBUYE YA QUE NECESITAMOS QUE SE ORDENEN PRIMERO

job.waitForCompletion()

La carpeta "output2" ya existe.


True

### EJ3
Todos los jugadores que usaron más de H héroes distintos (H es un parámetro de la
consulta).

In [ ]:
#Job 1: Acumulador de par (jugador, heroe)

inputPath = GENERALinputPath
outputPath = './output'
crearcarpeta("output")

def fmap(key, value, context):
  datos = value.split()

  for i in range(1,6):
    heroe = datos[i]
    context.write((key, datos[i]), "")

def freduce(key,values,context):
  for v in values:
    context.write(key,"")
    break

job = Job(inputPath, outputPath, fmap, freduce)
job.setCombiner(freduce) #EL COMBINER SI CONTRIBUYE => Solo transfiero un dato en vez de todos
job.waitForCompletion()

La carpeta "output" ya existe.


True

In [ ]:
#Job 2: Contador de repetición de jugador y solo da si cant > H

crearcarpeta("output2")
outputPath2 = './output2'

H = 20

inputPath = outputPath

def fmap(key, value, context):
  context.write(key, 1)

def freduce(key,values,context):
  cont = 0
  for v in values:
    cont+=v
  if(cont > context["H"]):
    context.write(key,cont)

def fcombiner(key,values,context):
  cont = 0
  for v in values:
    cont+=v
  context.write(key,cont)

job = Job(inputPath, outputPath2, fmap, freduce)

params = {"H": H}
job.setParams(params)

job.setCombiner(fcombiner) #EL COMBINER SI CONTRIBUYE => Contamos antes de trasnferir, por lo que trasferimos menos datos

job.waitForCompletion()

La carpeta "output2" ya existe.
2949


True

### EJ4
Todos los jugadores que participaron al menos N veces de la misión más jugada y el
puntaje total del jugador en esa misión sea mayor a P. La misión más jugada es aquella
en la que participaron más jugadores distintos, aunque sea una vez (N y P son
parámetros de la consulta). Suponga que la misión más jugada es única (no hay empate
en el primer puesto).


In [ ]:
#Job 1: Acumulador de par (jugador, mision)

inputPath = GENERALinputPath
outputPath = './output'
crearcarpeta("output")

def fmap(key, value, context):
  datos = value.split()

  context.write((key, datos[0]), "")

def freduce(key,values,context):
  for v in values:
    context.write(key,"")
    break

job = Job(inputPath, outputPath, fmap, freduce)
job.setCombiner(freduce) #EL COMBINER SI CONTRIBUYE => Solo transfiero un dato en vez de todos
job.waitForCompletion()

La carpeta "output" ya existe.


True

In [ ]:
#Job 2: Contador de repetición de misión

crearcarpeta("output2")
outputPath2 = './output2'

inputPath = outputPath

def fmap(key, value, context):
  context.write(value, 1)

def freduce(key,values,context):
  cont = 0
  for v in values:
    cont+=v
  context.write(key,cont)


job = Job(inputPath, outputPath2, fmap, freduce)
job.setCombiner(freduce) #EL COMBINER SI CONTRIBUYE => Contamos antes de trasnferir, por lo que trasferimos menos datos
job.waitForCompletion()

La carpeta "output2" ya existe.


True

In [ ]:
#Job 3: Obtenemos la mision mas jugada

crearcarpeta("output3")
outputPath3 = './output3'

inputPath = outputPath2

def fmap(key, value, context):
  context.write(1,(key,value))

def freduce(key,values,context):
 max=-1
 key= 0
 for v in values:
  cant=int(v[1])
  akey=v[0]
  if(cant>int(max)):
    max=cant
    bkey=akey
 context.write(bkey,max)


job = Job(inputPath, outputPath3, fmap, freduce)
job.setCombiner(freduce) #EL COMBINER SI CONTRIBUYE => Obtenemos el max parcial y solo trasferimos ese
job.waitForCompletion()

La carpeta "output3" ya existe.


True

In [ ]:
#Job 4: Jugadores con más de N partidas en la misión y puntaje acumulado mayor a P

crearcarpeta("output4")
outputPath4 = './output4'

N = 1
P = 5000

with open('output3/output.txt', 'r', encoding='utf-8') as f:
  for linea in f:
    elems = linea.strip().split()
    mision_con_max_cantidad = int(elems[0])
print(mision_con_max_cantidad)

inputPath = GENERALinputPath

def fmap(key, value, context):
  datos = value.split()
  if (int(datos[0]) == context["mision_con_max_cantidad"]):
    context.write(key, (1, int(datos[6])))

def freduce(key,values,context):
  cont = 0
  puntaje_acum = 0
  for v in values:
    cont+=v[0]
    puntaje_acum+=v[1]
  if (cont > context["N"] and puntaje_acum > context["P"]):
    context.write(key, (cont, puntaje_acum))

def fcombiner(key,values,context):
  cont = 0
  puntaje_acum = 0
  for v in values:
    cont+=v[0]
    puntaje_acum+=v[1]
  context.write(key, (cont, puntaje_acum))

job = Job(inputPath, outputPath4, fmap, freduce)

params = {"mision_con_max_cantidad": mision_con_max_cantidad,
          "N": N,
          "P": P}
job.setParams(params)
job.setCombiner(fcombiner) #EL COMBINER SI CONTRIBUYE => Obtenemos el max parcial y solo trasferimos ese

job.waitForCompletion()

Carpeta "output4" creada con éxito.
5


True